## Implement a simple network that can distinguish MNIST hand written digits

based on Michael Neilson's nn book: http://neuralnetworksanddeeplearning.com/chap1.html

In this notebook, we will try to modify the Network class to accept a different cost function, namely cross-entry cost function

\begin{eqnarray} 
  C = -\frac{1}{n} \sum_x \left[y \ln a + (1-y ) \ln (1-a) \right]
\end{eqnarray}

This looks complicated by when we compute dC/dw, it simplifies to avoid the sigmoid_prime term

\begin{eqnarray}
      \frac{\partial C}{\partial w^L_{jk}} & = & \frac{1}{n} \sum_x 
      a^{L-1}_k  (a^L_j-y_j) \\
      \frac{\partial C}{\partial b^L_{j}} & = & \frac{1}{n} \sum_x 
      (a^L_j-y_j).
\end{eqnarray}

The cross-entropy Cost function is introducted to avoid the slow down of learning caused by the near zero value of the derivative of sigmoid function when z is either very negative or very positive.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import pickle
import gzip
import random

In [3]:
# Load MNIST data set
def load_data(file_loc):
    with gzip.open(file_loc, 'rb') as file:
        training_data, validation_data, test_data = pickle.load(file, encoding='latin1')
    return (training_data, validation_data, test_data)

In [4]:
def vectorize_result(y):
    e = np.zeros((10,1))
    e[y] = 1.0
    return e

In [5]:
def load_mnist(file_loc):
    tr_data, val_data, test_data = load_data(file_loc)
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_data[0]]
    training_results = [vectorize_result(y) for y in tr_data[1]]
    training_data = list(zip(training_inputs, training_results))
    
    validation_inputs = [np.reshape(x, (784, 1)) for x in val_data[0]]
    validation_data = list(zip(validation_inputs, val_data[1]))
    
    test_inputs = [np.reshape(x, (784, 1)) for x in test_data[0]]
    test_data = list(zip(test_inputs, test_data[1]))
    return (training_data, validation_data, test_data)

In [6]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [20]:
class QuadraticCost:
    
    @staticmethod
    def fn():
        pass
    
    @staticmethod
    def delta(z, a, y):
        return (a - y) * sigmoid_prime(z)
    
class CrossEntropyCost:
    
    @staticmethod
    def fn():
        pass
    
    @staticmethod
    def delta(z, a, y):
        return (a - y)

class Network:
    
    def __init__(self, sizes, cost = QuadraticCost):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.weights = [np.random.randn(y, x) for x, y in list(zip(sizes[:-1], sizes[1:]))]
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.cost = cost
    
    def feed_fwd(self, a):
        for w, b in list(zip(self.weights, self.biases)):
            a = sigmoid(np.dot(w,a) + b)
        return a
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feed_fwd(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def SGD(self, train_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            n_test = len(test_data)
        n = len(train_data)
        for j in range(epochs):
            random.shuffle(train_data)
            mini_batches = [train_data[k: k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)}, {n_test}")
            else:
                print(f"Epoch {j}")
    
    def update_mini_batch(self, mini_batch, eta):
        m = len(mini_batch)
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        for x, y in mini_batch:
            # delta_nabla_w is nabla_w for each training example x derived using backprop
            delta_nabla_w, delta_nabla_b = self.back_prop(x, y)
            
            # then we accumulate the nablas contributed by all the training examples in the mini_batch in nabla_w
            nabla_w = [nw + dnw for nw, dnw in list(zip(nabla_w, delta_nabla_w))]
            nabla_b = [nb + dnb for nb, dnb in list(zip(nabla_b, delta_nabla_b))]
        self.weights = [w - (eta/m) * nw for w, nw in list(zip(self.weights, nabla_w))]
        self.biases  = [b - (eta/m) * nb for b, nb in list(zip(self.biases, nabla_b))]
    
    def back_prop(self, x, y):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        
        # feed fwd
        activations = [x]
        zs = []
        for w, b in list(zip(self.weights, self.biases)):
            z = np.dot(w, activations[-1]) + b
            zs.append(z)
            activations.append(sigmoid(z))
            
        # backward pass
        delta = self.cost.delta(zs[-1], activations[-1], y)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        nabla_b[-1] = delta
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
            nabla_b[-l] = delta
        return (nabla_w, nabla_b)

In [21]:
train_data, val_data, test_data = load_mnist('../data/mnist.pkl.gz')

In [22]:
import time

In [18]:
time1 = time.time() 
net = Network([784, 30, 10])
net.SGD(train_data, 10, 10, 3.0, test_data = test_data)
time2 = time.time()
print(f"Took {time2 - time1}")

Epoch 0: 8102, 10000
Epoch 1: 8320, 10000
Epoch 2: 9336, 10000
Epoch 3: 9339, 10000
Epoch 4: 9403, 10000
Epoch 5: 9424, 10000
Epoch 6: 9452, 10000
Epoch 7: 9436, 10000
Epoch 8: 9460, 10000
Epoch 9: 9465, 10000
Took 53.5204873085022


Note from Michael Nielson

There is, incidentally, a very rough general heuristic for relating the learning rate for the cross-entropy and the quadratic cost. As we saw earlier, the gradient terms for the quadratic cost have an extra σ′=σ(1−σ) term in them. Suppose we average this over values for σ, ∫10dσσ(1−σ)=1/6. We see that (very roughly) the quadratic cost learns an average of 6 times slower, for the same learning rate. This suggests that a reasonable starting point is to divide the learning rate for the quadratic cost by 6. Of course, this argument is far from rigorous, and shouldn't be taken too seriously. Still, it can sometimes be a useful starting point.

In [24]:
time1 = time.time() 
net = Network([784, 30, 10], cost=CrossEntropyCost)
net.SGD(train_data, 10, 10, .5, test_data = test_data) # Set the learning rate lower
time2 = time.time()
print(f"Took {time2 - time1}")

Epoch 0: 9153, 10000
Epoch 1: 9251, 10000
Epoch 2: 9280, 10000
Epoch 3: 9358, 10000
Epoch 4: 9357, 10000
Epoch 5: 9385, 10000
Epoch 6: 9404, 10000
Epoch 7: 9425, 10000
Epoch 8: 9452, 10000
Epoch 9: 9455, 10000
Took 49.707035064697266
